# Prediction of house price - [Kaggle](https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/) competition

## Intro

Main purpose of this project is to create regression model for prediction of house price. 

### Import necessary libraries

In [166]:
# general libs
from datetime import datetime
import os
import glob
from itertools import cycle

# data analysis libs
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.tools import make_subplots


#ML
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb


### Read train data and take first look on train data

In [125]:
all_files = glob.glob("train.csv")
df = pd.concat((pd.read_csv(f) for f in all_files))
print(df.shape)
train_size=df.shape[0]
price=df['SalePrice']

(1460, 81)


### Read and merge train and test data

In [126]:
all_files = glob.glob("[t]?*.csv")
# .csv files downloaded from binance.com, and contains information about pair BTC-USDT with frequency 15 minutes from 21.0.22 to 23.09.22
# load multiple .csv files to dataframe
all_files.sort(reverse=True)
print(all_files)
df = pd.concat((pd.read_csv(f) for f in all_files))
# remove irrelevant ID column from dataset
df.drop('Id',axis=1, inplace=True)
df.info()
df.head()

['train.csv', 'test.csv']
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2919 entries, 0 to 1458
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     2919 non-null   int64  
 1   MSZoning       2915 non-null   object 
 2   LotFrontage    2433 non-null   float64
 3   LotArea        2919 non-null   int64  
 4   Street         2919 non-null   object 
 5   Alley          198 non-null    object 
 6   LotShape       2919 non-null   object 
 7   LandContour    2919 non-null   object 
 8   Utilities      2917 non-null   object 
 9   LotConfig      2919 non-null   object 
 10  LandSlope      2919 non-null   object 
 11  Neighborhood   2919 non-null   object 
 12  Condition1     2919 non-null   object 
 13  Condition2     2919 non-null   object 
 14  BldgType       2919 non-null   object 
 15  HouseStyle     2919 non-null   object 
 16  OverallQual    2919 non-null   int64  
 17  OverallCond    2919 non-nu

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500.0
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500.0
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500.0
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000.0
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000.0


### Data cleaning

In [127]:
print(df.shape)
#check for NaN values
df.isnull().sum()[df.isnull().sum()>0]

(2919, 80)


MSZoning           4
LotFrontage      486
Alley           2721
Utilities          2
Exterior1st        1
Exterior2nd        1
MasVnrType        24
MasVnrArea        23
BsmtQual          81
BsmtCond          82
BsmtExposure      82
BsmtFinType1      79
BsmtFinSF1         1
BsmtFinType2      80
BsmtFinSF2         1
BsmtUnfSF          1
TotalBsmtSF        1
Electrical         1
BsmtFullBath       2
BsmtHalfBath       2
KitchenQual        1
Functional         2
FireplaceQu     1420
GarageType       157
GarageYrBlt      159
GarageFinish     159
GarageCars         1
GarageArea         1
GarageQual       159
GarageCond       159
PoolQC          2909
Fence           2348
MiscFeature     2814
SaleType           1
SalePrice       1459
dtype: int64

In [128]:
# drop all columns where number of NaN is more that half of the dataset
df.drop(df.isnull().sum()[df.isnull().sum()>0].index[df.isnull().sum()[df.isnull().sum()>0]>df.shape[0]/2],axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2919 entries, 0 to 1458
Data columns (total 76 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     2919 non-null   int64  
 1   MSZoning       2915 non-null   object 
 2   LotFrontage    2433 non-null   float64
 3   LotArea        2919 non-null   int64  
 4   Street         2919 non-null   object 
 5   LotShape       2919 non-null   object 
 6   LandContour    2919 non-null   object 
 7   Utilities      2917 non-null   object 
 8   LotConfig      2919 non-null   object 
 9   LandSlope      2919 non-null   object 
 10  Neighborhood   2919 non-null   object 
 11  Condition1     2919 non-null   object 
 12  Condition2     2919 non-null   object 
 13  BldgType       2919 non-null   object 
 14  HouseStyle     2919 non-null   object 
 15  OverallQual    2919 non-null   int64  
 16  OverallCond    2919 non-null   int64  
 17  YearBuilt      2919 non-null   int64  
 18  YearRemo

In [129]:
# replace NaN by mode for non-digital value 
for column in df.columns[df.dtypes=="O"]:
    df[column]=df[column].fillna(df[column].mode()[0])
# and mean for digital    
for column in df.columns[df.dtypes!="O"]:
    df[column]=df[column].fillna(df[column].mean())

In [130]:
# check if there is still NaN
df.isnull().sum()[df.isnull().sum()>0]

Series([], dtype: int64)

In [131]:
# extract features from text column
new_df=df
for column in df.columns[df.dtypes=="O"]:
    new_df=pd.concat([new_df, pd.get_dummies(data=df[column], columns=[column], drop_first=True)], axis=1)

In [132]:
# delete this column
for column in df.columns[df.dtypes=="O"]:
    new_df.drop(column,axis=1, inplace=True)

In [133]:
new_df.isnull().sum()[new_df.isnull().sum()>0]

Series([], dtype: int64)

### Split data on test and train

In [134]:
train_data=new_df.iloc[:train_size,:].copy()
test_data=new_df.iloc[train_size:,:].copy()
y_train=train_data['SalePrice']
print(train_data.shape)
print(test_data.shape)
train_data.drop('SalePrice',axis=1, inplace=True)
test_data.drop('SalePrice',axis=1, inplace=True)


(1460, 237)
(1459, 237)


### Some plots

In [176]:
separation_by_year=new_df.groupby('YearBuilt')
years = [pair for pair, dat_fr in separation_by_year]

# Create figure with secondary y-axis
fig = go.Figure()
#fig = make_subplots(specs=[[{"secondary_y": True}]])
# Add traces

fig.add_trace(
    go.Bar(x=years, y=separation_by_year.mean()['OverallQual'], name='Overall Quality')
    #,secondary_y=False
)

fig.add_trace(
    go.Bar(x=years, y=separation_by_year.mean()['OverallCond'], name="Overal Condition")
    #, secondary_y=True
)

# Add figure title
fig.update_layout(
    title_text="Overal Quality and Condition by year", plot_bgcolor='white'
)

# Set x-axis title
fig.update_xaxes(title_text="Year")
# Set y-axis title
fig.update_yaxes(title_text="Rating")

# Set y-axes titles
#fig.update_yaxes(title_text="Quality Rating", secondary_y=False)
#fig.update_yaxes(title_text="Condition Rating", secondary_y=True)

fig.show()

In [177]:
separation_by_year=new_df.groupby('YearRemodAdd')
years = [pair for pair, dat_fr in separation_by_year]

# Create figure with secondary y-axis
fig = go.Figure()
#fig = make_subplots(specs=[[{"secondary_y": True}]])
# Add traces

fig.add_trace(
    go.Bar(x=years, y=separation_by_year.mean()['OverallQual'], name='Overall Quality')
    #,    secondary_y=False
)

fig.add_trace(
    go.Bar(x=years, y=separation_by_year.mean()['OverallCond'], name="Overal Condition")
    #,    secondary_y=True
)

# Add figure title
fig.update_layout(
    title_text="Overal Quality and Condition by year",  plot_bgcolor='white'
)

# Set x-axis title
fig.update_xaxes(title_text="Year")
# Set y-axis title
fig.update_yaxes(title_text="Rating")

# Set y-axes titles
#fig.update_yaxes(title_text="Quality Rating", secondary_y=False)
#fig.update_yaxes(title_text="Condition Rating", secondary_y=True)

fig.show()

### Create  extreme gradient boost regressor

In [86]:
xg_reg = xgb.XGBRegressor()


booster=['gbtree','gblinear']
base_score=[0.25,0.5,0.75,1]

## Hyper Parameter Optimization


n_estimators = [50,100, 500, 900, 1100, 1500]
max_depth = [2, 3, 5, 10, 15]
booster=['gbtree','gblinear']
learning_rate=[0.05,0.1,0.15,0.20]
min_child_weight=[1,2,3,4]
colsample_bytree=[0.2,0.3,0.4,0.5,0.8, 1]

# Define the grid of hyperparameters to search
hyperparameter_grid = {
    'n_estimators': n_estimators,
    'max_depth':max_depth,
    'learning_rate':learning_rate,
    'min_child_weight':min_child_weight,
    'booster':booster,
    'base_score':base_score,
    'colsample_bytree':colsample_bytree
    }



# Set up the random search with 4-fold cross validation
random_cv = RandomizedSearchCV(estimator=xg_reg,
            param_distributions=hyperparameter_grid,
            cv=5, n_iter=50,
            scoring = 'neg_mean_absolute_error',n_jobs = 4,
            verbose = 5, 
            return_train_score = True,
            random_state=42)



In [87]:

random_cv.fit(train_data.values,y_train.values)


Fitting 5 folds for each of 50 candidates, totalling 250 fits


RandomizedSearchCV(cv=5,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None,
                                          enable_categorical=False, gamma=None,
                                          gpu_id=None, importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=None,
                                          max_delta_step=None, max_depth=None,
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=...
                   n_iter=50, n_jobs=4,
                   param_distributions={'base_score': [0.25, 0.5, 0.75, 1],
                                        'booster': ['gbtree', 'gblinear'],
                                        'colsample_bytree': [0.2, 0.3, 0.4, 0.5,
                                                             0.8, 1],
                                        'learning_rate': [0.05, 0.1, 0.15, 0.2],
                                        'max_depth': [2, 3, 5, 10, 15],
                                        'min_child_weight': [1, 2, 3, 4],
                                        'n_estimators': [50, 100, 500, 900,
                                                         1100, 1500]},
                   random_state=42, return_train_score=True,
                   scoring='neg_mean_absolute_error', verbose=5)

In [88]:


random_cv.best_estimator_



XGBRegressor(base_score=0.25, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.5, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.15, max_delta_step=0,
             max_depth=2, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=1500, n_jobs=16,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [90]:
xg_reg = xgb.XGBRegressor(alpha=10, base_score=0.25, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.5, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.15, max_delta_step=0,
             max_depth=2, min_child_weight=1, monotone_constraints='()', n_estimators=1500, n_jobs=16,
             num_parallel_tree=1, predictor='auto', random_state=0,
             reg_alpha=10, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)
xg_reg.fit(train_data.values,y_train.values)



import pickle
filename = 'finalized_model.pkl'
pickle.dump(xg_reg, open(filename, 'wb'))



preds = xg_reg.predict(train_data.values)




In [101]:
preds = xg_reg.predict(test_data.values)

In [109]:
##Create Sample Submission file and Submit
pred=pd.DataFrame(preds)
sub_df=pd.read_csv('sample_submission.csv')
datasets=pd.concat([sub_df['Id'],pred],axis=1)
datasets.columns=['Id','SalePrice']
datasets.info()
datasets.tail()

datasets.Id = datasets.Id.astype(int)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Id         1459 non-null   int64  
 1   SalePrice  1459 non-null   float32
dtypes: float32(1), int64(1)
memory usage: 17.2 KB


In [110]:
datasets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Id         1459 non-null   int32  
 1   SalePrice  1459 non-null   float32
dtypes: float32(1), int32(1)
memory usage: 11.5 KB


In [111]:
datasets.to_csv('sample_submission.csv',index=False)
datasets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Id         1459 non-null   int32  
 1   SalePrice  1459 non-null   float32
dtypes: float32(1), int32(1)
memory usage: 11.5 KB
